In [18]:
import numpy as np; import pandas as pd; 
import sys; import importlib;
sys.path.append('/media/mydrive/PhysOc/modview/modview')
import phystools; import xarray as xr
import time

In [19]:
importlib.reload(phystools);

Let's test some of properties and methods of the "internal_wave" class in modview/phystools. To begin, we create two class instances, one for a near-inertial wave, and another for an internal tide. 

In [20]:
phi_0 = 28.9; # initial latitude

niw = phystools.internal_wave(lat=phi_0, N2 = 1e-5); # near-inertial wave
niw.fk = {'omega':1.1*phystools.inertial_freq(niw.lat),'kx':2*np.pi/10e5,
                      'ky':2*np.pi/15e4,'kz':np.nan};

itw = phystools.internal_wave(lat=phi_0, N2 = 1e-5); # M2 internal tide
itw.fk = {'omega':2*np.pi/(12.37*3600),'kx':2*np.pi/2e4,'ky':-2*np.pi/1e4,'kz':np.nan}

Missing parameter kz= 2*pi/-1511.46 added successfully
Missing parameter kz= 2*pi/-346.37 added successfully


Now that two instances of the internal wave class have been created successfully (and their phase parameters specified), we can compute terms in their dispersion relation and evaluate their group velocity in all directions. 

In [21]:
start_time = time.time()
print(itw.group_vel(direction=['x','y','z'], numeric=True))
print("--- %s seconds ---" % (time.time() - start_time))

[1.90276233721589e-5, -3.80552467443179e-5, 1.64764968423908e-6]
--- 0.0014829635620117188 seconds ---


In [22]:
nx = 30; ny = 10; nz = 15;
fk_coords = {'lon':np.linspace(140,150,nx), 
               'lat':np.linspace(20,30,ny),
              'p':np.linspace(0,300, nz)}

def cube_generator(coords, gradients, meanval):
    # Create axes and use those to span linear gradients in a volume
    x = np.expand_dims(coords['lon'],axis=(1,2)); 
    x = x - np.nanmean(x);
    y = np.expand_dims(coords['lat'],axis=(0,2));
    y = y - np.nanmean(y);
    z = np.expand_dims(coords['p'],axis=(0,1)); 
    z = z - np.nanmean(z); 
    
    cube = meanval + x*gradients[0] + y*gradients[1] + z*gradients[2];
    return cube
# Fake temperature data
CT_grads = [1/40, 1/4, -5/100]
fk_CT = cube_generator(fk_coords, CT_grads, 20)
fk_CT = xr.DataArray(data=fk_CT, dims=['lon','lat','p'],
                    coords=fk_coords)
# Fake salinity data
SAL_grads = [0,1/10,5/1000]
fk_SAL = cube_generator(fk_coords, SAL_grads, 33);
fk_SAL = xr.DataArray(data=fk_SAL, dims=['lon','lat','p'],
                     coords=fk_coords)
fk_CT = np.squeeze(fk_CT); 
fk_SAL = np.squeeze(fk_SAL);

In [23]:
myfield = phystools.geostrophic_field(fk_CT.transpose(),
                                      fk_SAL.transpose(), fk_CT.p)

In [24]:
myfield.streamfunction = 'fds'

In [25]:
myfield.streamfunction.sel(lon=145,method='nearest').plot()

AttributeError: 'str' object has no attribute 'sel'

In [149]:
myfield.density

<xarray.DataArray (p: 15, lat: 10, lon: 30)>
array([[[1020.47615094, 1020.4735121 , 1020.4708727 , ...,
         1020.40470251, 1020.40204831, 1020.39939354],
        [1020.47273681, 1020.47007783, 1020.46741828, ...,
         1020.40074552, 1020.39807125, 1020.39539642],
        [1020.46861908, 1020.46594006, 1020.46326048, ...,
         1020.3960878 , 1020.39339358, 1020.39069879],
        ...,
        [1020.43760453, 1020.43482689, 1020.4320487 , ...,
         1020.3624159 , 1020.35962347, 1020.35683049],
        [1020.4293417 , 1020.42654465, 1020.42374705, ...,
         1020.35363   , 1020.35081824, 1020.34800594],
        [1020.42040052, 1020.41758416, 1020.41476726, ...,
         1020.34416852, 1020.34133753, 1020.338506  ]],

       [[1020.97036025, 1020.96778872, 1020.9652166 , ...,
         1020.90072541, 1020.89813824, 1020.89555049],
        [1020.96932418, 1020.96673214, 1020.96413954, ...,
         1020.89913695, 1020.89652937, 1020.8939212 ],
        [1020.96757221, 1020.96495979, 1020.96234679, ...,
         1020.89683551, 1020.89420762, 1020.89157915],
...
        [1026.43876537, 1026.43692035, 1026.43507465, ...,
         1026.38870876, 1026.38684521, 1026.38498098],
        [1026.46349637, 1026.46162701, 1026.45975695, ...,
         1026.41278376, 1026.41089597, 1026.4090075 ],
        [1026.48737467, 1026.4854811 , 1026.48358685, ...,
         1026.43600998, 1026.4340981 , 1026.43218554]],

       [[1026.62135543, 1026.61976925, 1026.61818232, ...,
         1026.57827133, 1026.57666539, 1026.57505871],
        [1026.65518637, 1026.65357422, 1026.65196134, ...,
         1026.61140301, 1026.60977124, 1026.60813874],
        [1026.68810799, 1026.68647005, 1026.68483139, ...,
         1026.64362991, 1026.64197247, 1026.64031431],
        ...,
        [1026.8392798 , 1026.83751535, 1026.8357502 , ...,
         1026.7913938 , 1026.78961045, 1026.78782641],
        [1026.86686653, 1026.86507725, 1026.86328727, ...,
         1026.81831167, 1026.81650361, 1026.81469486],
        [1026.89358358, 1026.89176962, 1026.88995498, ...,
         1026.84436398, 1026.84253136, 1026.84069806]]])
Coordinates:
  * lon      (lon) float64 140.0 140.3 140.7 141.0 ... 149.0 149.3 149.7 150.0
  * lat      (lat) float64 20.0 21.11 22.22 23.33 ... 26.67 27.78 28.89 30.0
  * p        (p) float64 0.0 21.43 42.86 64.29 85.71 ... 235.7 257.1 278.6 300.0